# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846806406657                   -0.70    4.5
  2   -7.852310407361       -2.26       -1.53    1.0
  3   -7.852646183436       -3.47       -2.52    3.2
  4   -7.852646677465       -6.31       -3.36    2.2
  5   -7.852646685839       -8.08       -4.75    1.2
  6   -7.852646686722       -9.05       -5.17    3.8
  7   -7.852646686729      -11.13       -6.00    1.2
  8   -7.852646686730      -12.37       -7.23    3.0
  9   -7.852646686730   +    -Inf       -7.43    2.0
 10   -7.852646686730   +  -14.57       -8.64    2.2
 11   -7.852646686730      -14.45       -9.29    2.8
 12   -7.852646686730   +  -14.57       -9.65    1.0
 13   -7.852646686730      -14.75      -11.09    2.2
 14   -7.852646686730   +  -14.45      -11.48    2.5
 15   -7.852646686730      -14.45      -11.86    1.0
 16   -7.852646686730      -14.75      -12.99    2.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846780735059                   -0.70           4.5
  2   -7.852528717365       -2.24       -1.63   0.80    2.2
  3   -7.852637335546       -3.96       -2.72   0.80    1.0
  4   -7.852646478633       -5.04       -3.30   0.80    2.2
  5   -7.852646668918       -6.72       -4.16   0.80    1.2
  6   -7.852646686406       -7.76       -4.74   0.80    2.0
  7   -7.852646686722       -9.50       -5.44   0.80    1.8
  8   -7.852646686729      -11.14       -6.96   0.80    1.8
  9   -7.852646686730      -12.24       -7.44   0.80    3.2
 10   -7.852646686730      -14.45       -8.18   0.80    1.0
 11   -7.852646686730   +  -14.75       -8.90   0.80    2.2
 12   -7.852646686730      -14.75      -10.08   0.80    1.5
 13   -7.852646686730   +  -14.45      -10.09   0.80    2.8
 14   -7.852646686730      -14.35      -11.33   0.80    1.0
 15   -7.852646686730   +  -14.57      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.390393e+01     4.287703e+00
 * time: 0.4995279312133789
     1     1.390319e+00     2.163428e+00
 * time: 0.7428591251373291
     2    -1.221357e+00     2.645623e+00
 * time: 0.7794439792633057
     3    -3.698531e+00     1.838982e+00
 * time: 0.8236279487609863
     4    -5.158562e+00     1.288749e+00
 * time: 0.981097936630249
     5    -6.757979e+00     7.212761e-01
 * time: 1.016834020614624
     6    -7.037047e+00     8.678223e-01
 * time: 1.0424089431762695
     7    -7.438819e+00     4.668649e-01
 * time: 1.0689170360565186
     8    -7.566091e+00     3.799257e-01
 * time: 1.0967421531677246
     9    -7.654516e+00     1.459314e-01
 * time: 1.1224370002746582
    10    -7.749618e+00     2.081613e-01
 * time: 1.14888596534729
    11    -7.784286e+00     1.004705e-01
 * time: 1.1759650707244873
    12    -7.818261e+00     7.277526e-02
 * time: 1.2054691314697266
    13    -7.837797e+00     7.424505e-02
 * time: 1.23461699485778

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846873040182                   -0.70    4.5
  2   -7.852328673684       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686715                   -2.55
  2   -7.852646686730      -10.82       -6.06
  3   -7.852646686730   +    -Inf      -12.70


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.2244577738934477e-14
|ρ_newton - ρ_scfv| = 1.4888816236277857e-12
|ρ_newton - ρ_dm|   = 6.129512244043496e-10
